In [1]:
import numpy as np
from gurobipy import *


# Steady State Formulation 

**Example of Formulation using GurobiPy**
http://www.gurobi.com/resources/examples/food-manufacture-I

In [2]:
# change all Hayward to Warm Springs and set a constraint where we cannot make / break at that area. 
# How do we do this as a constraint? 
# NO M/B at Hayward/WarmSprings, Concord or SFO. Have this as a constraint 


## Data  

**How do we initialize according to a future schedule. What do we tell them to do in terms of train lengths. Assuming we are given a schedule.**


**To Do**

- Add the Santa Clara extension as a segment? No.
- Add the Santa Clara extension as a yard with inventory? No.  
- Input the additional data that we receive from Sandy
- Account for unbreakable trains in model. Nope (reverse those changes)
    - Have separate inventory.
    - Separate inventory for breakable trains will coincide with the total inventory of 10 car trains at each yard.
        - Making will occur only when there are no 10 car trains available (unbreakable or breakable)
        - Breaking will occur only when there is a breakable train at that yard
- Add constraints to ensure no making/breaking at end of line locations that are not yards. 
- Initialize model with the schedule
    - The Schedule is a set of (departure time, arrival time, segment)
- Do we need a minimum threshold to ensure that making/breaking is always possible? 
- Input the one way travel time data to the model
- Need constraint to account for time to m/b + aux cycle + factor in for expected turn around time from data? 


**The Steady State Assumptions**

- 1081 D, E cars (all 2, 3 composition) 
   - All breakable 
- Use current system map
- No unbreakable consists

- New interpretation. To be implemented. 
    - c represents the ith departure (an index)
        - This index can retrieve alpha_{segment, yard, index} = arrival time and delta_{segment, yard, index} = departure time 
- Inventory long/short train fulfillment is always preferable to a make / break
        
        
        
**Issues**
- Consists need to be able to get to inventory points like concord that are not explicitly part of the segments because they are not on an end of the line...
    - We assume yards that are not at the end of the line never do m/b cause they still have passengers onboard 
    - Does this mean we should omit them entirely from our model cause they aren't at the end of a line? 
    
    - THERE ISNT A TRAVEL SEGMENT TO GET TO SOME INVENTORY POINTS CAUSE THEY'RE NOT AT THE END OF A LINE...
        - For now get rid of them? When we get the mile data we can input correct distance miles and what not. 
           - Unknown. Added concord back in but thats probably not appropriate. 


In [10]:
## Sets

# Travel segments A
segment = np.arange(1, 13)

# Day B. Model for weekdays only. 
day = ["Mon", "Tues", "Wed", "Thurs", "Fri"]


# Index represents the ith departure from the start of the train segment to the end of the train segment. 
# Assume there are no more than 500 departures on a line segment for any given day. 
index = np.arange(500)


# Read SCRAMS Data into this format. Dao will format. 
# Dictionary representing the arrival time of a consist on a segment, for a given day, for a given index. 
alpha = {}

# Read SCRAMS Data into this format. Dao will format. 
# Dictionary representing the departure time of a consist on a segment, for a given day, for a given index. 
delta = {}


## Set of departure and arrival times and the given day. From the SCRAM schedule. 
# Check that set for inventory balance requirements only rather than check every time. 
# train segment, day, time in ( (Richmond-Daly City, Monday, 0812), (Daly City - Richmond, Wed, 1917)... ) 
# As opposed to train segment in (Richmond-Daly City, Daly City- Richmond ...), time in (0000-2359)


# Yard / Tail Tracks D (Some of these will not have m/b capacity. Determined by constraints)
# Remove Concord from this list as it is not an end of line yard. Assume that its trains will depart from Pittsburg. 
yard_tt = ["Richmond", "Daly City", "WarmSprings", "Milbrae", "East Dublin", "Pittsburg", "SFO"]
no_makebreak_yards = ["WarmSprings, SFO"]


# Sequence of Train Segments E 
# line = {"g1" : [12], 
#     "g2": [11], 
#     "o1": [3], 
#     "o2": [4],
#     "y1": [9], 
#     "y2": [10], 
#     "r1": [1, 5], 
#     "r2": [6, 2], 
#     "b1": [7],
#     "b2": [8]}

# Shift s. 19 Hours of Operation. Split this into 2 hour shifts. 9.5 shifts. 10 shifts. 
shift = np.arange(1, 11)
    
# Type of train t
# 5 = 5 car consist
# 10 =  breakable 10 car consist composed of two 5 car trains: 4 D, 6 E
# 100 = unbreakable 10 car consist composed of: 2 D, 8 E
typeTrain = [5, 10]

# Segment to Yard/TT relation dictionary. 
segmentYard_Relation = {1 : ["Richmond", "Milbrae"], 
                       2: ["Milbrae", 'Richmond'], 
                       3: ["Richmond", "WarmSprings"], 
                       4: ["WarmSprings", "Richmond"], 
                       5: ["Daly City", "East Dublin"], 
                       6: ["East Dublin", "Daly City"], 
                       7: ["Pittsburg", "Daly City"], 
                       8: ["Daly City", "Pittsburg"],
                       9: ["Daly City", "WarmSprings"], 
                       10: ["WarmSprings", "Daly City"],
                       11: ["Pittsburg", "SFO"], 
                       12: ["SFO", "Pittsburg"]}
#                        11: ["Pittsburg", "Concord"], 
#                        12: ["Concord", "SFO"],
#                        13: ["SFO", "Concord"],
#                        14: ["Concord", "Pittsburg"]}
    
    

## Parameters

# energy cost per car mile. Assume C_D = C_E = C
energy_Cost = 0

# maintenance per car mile. Assume M_D = M_E = M
# Assuming some steady maintenance cost per mile traveled. 
maintenance_Cost = 0

# Energy req to run D/E train car per car mile 
energy_Mile = {"D": 0,
               "E": 0}


# Cost of conducting M/B 
cost_MakeBreak = 0

# Hourly wear and tear cost of operating a train_type
wearTear_Hour = {"D": 0, 
                 "E": 0}

# Hourly wage of yard operators. Are they paid hourly? (Assumption Based off GlassDoor + BART 2019 Payroll, Refine more Later)
# hourlyWage = 32

# Spare Ratio. Justify that we're not using this in the report.
# Do this as a post processing step. Check at the end. Makes notes of this. 
spareRatio = 0.2

# Total number of cars that inventory can hold in the system. Determined by total storage capacity. (Typically there is a system wide average capacity fill of 80 percent though)
# totalCars = 2026

# Bad Cars percentage in the system


# Total number of 5 car trains that can be held in inventory at yard_tt.
# At Start and at end of Day
# yardInventory_5 = {
#     "Concord": 9 + 19 * 2, 
#     "Daly": 3 + 12 * 2, 
#     "Hayward": 4 + 28 * 2, 
#     "Richmond": 7 + 16 * 2, 
#     "Dublin": 0 + 7 * 2,
#     "Milbrae": 3 + 4 * 2,
#     "Pittsburg": 0 + 2 * 2
# }

yardInventory_5 = {
#     "Concord": 9, 
    "Daly City": 3, 
    "WarmSprings": 4, 
    "Richmond": 7, 
    "East Dublin": 0,
    "Milbrae": 3,
    "Pittsburg": 0, 
    "SFO": 0
}


# Total number of 10 car trains that can be held in inventory at yard_tt. 
# At Start and at end of Day
yardInventory_10 = {
#     "Concord": 19, 
    "Daly City": 12, 
    "WarmSprings": 28, 
    "Richmond": 16, 
    "East Dublin": 7,
    "Milbrae": 4,
    "Pittsburg": 2,
    "SFO": 0
}


# Inventory. Number of cars at yard_tt Dat time C. yard, time, (Space for 5, Space for 10)
# inventoryYard = {}
# for yard in yard_tt:
#     for t in time:
#         for length in typeTrain:
#             # Filler Entry for each possible permutation
#             inventoryYard[(yard, length, t)] = 0


            
            
# Maximum Carrying Capacity of D Car
cc_d = 187

# Maximum Carrying Capacity of E Car
cc_e = 191


# Number of passengers forecasted for line supported by yard_tt at a time, and day.
# What would this data look like? Give a demand to meet per train segment? 
# Idea: Constraint TS_abc to be 1 or 0 based off of necesity to meet the passenger demand. 
# PREPROCESSING: Assume we're given a max amount of passengers that need to be able to get on a train on a given line. This can remove some DV.
# Constraint: Assume we're given a max amount of passengers (for one of the stations on the line) that need to be able to get on a train on a given line.
#
# How to input the RHS of the constraint into the model. 
# What form? For each train segment, total number of passengers that are going one direction on the line for a given hour. 
# Our constraint will make sure that we have enough trains to meet this demand on the line direction for an hour. 
# This is an aggregate approach to the problem. 

# For each line segment, for each time, for each day. 
# Value = the maximum number of passengers that could be on a train (given demand) on a line.
# = Number of people that want to be on the train at the busiest part of the segment. 
P_abc = {1:0,
           2:0,
           3:0,
           4:0,
           5:0,
           6:0,
           7:0,
           8:0,
           9:0,
           10:0,
           11:0,
           12:0}
#            13:0, 
#            14:0}


# Rail distance of travel segment a. Dictionary. 
railDistance = {1:0,
           2:0,
           3:0,
           4:0,
           5:0,
           6:0,
           7:0,
           8:0,
           9:0,
           10:0,
           11:0,
           12:0}
#            13:0, 
#            14:0}

# One wy travel time for travel segment a. Dictionary. 
# Need distance to get proportion to be able to compute this. 
owTravel = {1:69,
           2:69,
           3:62,
           4:62,
           5:64,
           6:64,
           7:71,
           8:71,
           9:71,
           10:71,
           11:88,
           12:88} # Pittsburg - SFO = 88 minutes. 
#            11:11, # Filler Value: 88 * 1/8 for Pitt to Cord
#            12:77, # Filler Value
#            13: 77, # Filler Value
#            14: 11} # Filler Value 

# Cost of hiring an additional yard team for shift s. 
# 2 people, 2 hours at a rate of 32 USD / hour
costYardTeam_shift = {1:2*2*32, 
                     2:128,
                     3:128,
                     4:128,
                     5:128,
                     6:128,
                     7:128,
                     8:128,
                     9:128,
                     10:128}


## Levels of operations. What would these be set by? 
# These just determine the maxium capacity (how many makes/breaks they can do). 
# Set one level for each yard. Run 3 times to test them. 
# Run 3 times, adjust threshold in the constraint & adjust the cost that it will infer on the objective function. 
# In this case there would be no DV. 
# - Low Case - Less workers than current operations. 
# - Base case (current operations)
# - High Case - Additional workers in a shift 

# Just test the 3 cases? Dont need to write them in as DV. 



## Model

### Decision Variables

In [4]:
# Create a new model
model = Model('steady state fleet')

# Create variables
# Binary Variable. Type of train from travel segment a, at departure time b, day c
segmentTrain = model.addVars(segment, day, index, vtype=GRB.BINARY, name="segmentTrain")

# Binary Variable. Level of staff needed at yard y and shift s
levelStaff = model.addVars(yard_tt, shift, vtype=GRB.BINARY, name="levelStaff")

model.update()

# AUXILIARY VARIABLE NOT DV. Inventory Variable. Balance at yard_tt, of specified length, at a given time. 
inventoryYard = model.addVars(yard_tt, typeTrain, index, name="inventoryYard")
model.update()

Academic license - for non-commercial use only


### Objective Function

In [5]:
obj = ((2 * energy_Mile.get("D") + 3 * energy_Mile.get("E")) * energy_Cost 
       * sum([(1 + segmentTrain.get((a, b, c))) * railDistance.get(a) for a, b, c in [(a, b, c) for a in segment for b in day for c in index]])
      
      + sum([levelStaff.get((y, s)) * costYardTeam_shift.get(s) for y, s in [(y, s) for y in yard_tt for s in shift]])
      
      + cost_MakeBreak * sum([(segmentTrain.get((a, b, c)) * (1 if (inventoryYard.get((y, 10, c)) < 1 and inventoryYard.get((y, 5, c)) >= 2) else 0) 
                      + (1 - segmentTrain.get((a, b, c))) * (1 if (inventoryYard.get((y, 5, c)) < 1 and inventoryYard.get((y, 10, c) >= 1)) else 0)) 
                      for a, b, c, y in [(a, b, c, y) 
                                         for a in segment if segmentYard_Relation.get(a)[0] == y 
                                         for b in day 
                                         for c in index 
                                         for y in yard_tt]])    
      + (2 * wearTear_Hour.get("D") + 3 * wearTear_Hour.get("E")) * maintenance_Cost 
      * sum([(1 + segmentTrain.get((a, b, c))) * railDistance.get(a) for a, b, c in [(a, b, c) for a in segment for b in day for c in index]]))

In [6]:
model.setObjective(obj, GRB.MINIMIZE)

### Constraints


In [7]:
# Testing stuff


In [8]:
# Meet Carrying Capacity (DEMAND)
# ?


In [10]:
# Run trains across all lines E.
# ?


In [9]:
# Inventory Non Negativity. Binary Nature of Other DV set when initialized.
model.addConstrs((inventoryYard.get((yard, l, t)) >= 0 for yard, l, t in [(yard, l, t) for yard in yard_tt for l in typeTrain for t in index]))
0

0

In [12]:
# For first departure of the day 
model.addConstrs((inventoryYard.get((yard, 5, 0)) <= yardInventory_5.get(yard) for yard in yard_tt))

# For last departure of the day?

# model.addConstrs((inventoryYard.get((yard, 5, 300)) <= yardInventory_5.get(yard) for yard in yard_tt))
# model.addConstrs((inventoryYard.get((yard, 5, 1440)) <= yardInventory_5.get(yard) for yard in yard_tt))
# model.addConstrs((inventoryYard.get((yard, 10, 300)) <= yardInventory_10.get(yard) for yard in yard_tt))
# model.addConstrs((inventoryYard.get((yard, 10, 1440)) <= yardInventory_10.get(yard) for yard in yard_tt))
# 0

{'Daly City': <gurobi.Constr *Awaiting Model Update*>,
 'East Dublin': <gurobi.Constr *Awaiting Model Update*>,
 'Milbrae': <gurobi.Constr *Awaiting Model Update*>,
 'Pittsburg': <gurobi.Constr *Awaiting Model Update*>,
 'Richmond': <gurobi.Constr *Awaiting Model Update*>,
 'SFO': <gurobi.Constr *Awaiting Model Update*>,
 'WarmSprings': <gurobi.Constr *Awaiting Model Update*>}

In [13]:
# # Inventory at start and end of the day must be below or meet inventory capacity. 
# for yard in yard_tt:
#     for t in typeTrain:
#         # 300 is start of day
#         # 1440 is end of day
#         # When both sets of constraints are introduced below the model is infeasible.Just one. Ok. 
#         if t == 5:
#             model.addConstr(inventoryYard.get((yard, t, 300)) <= yardInventory_5.get(yard))
#             model.addConstr(inventoryYard.get((yard, t, 1440)) <= yardInventory_5.get(yard))
#         if t == 10: 
#             model.addConstr(inventoryYard.get((yard, t, 300)) <= yardInventory_10.get(yard))
#             model.addConstr(inventoryYard.get((yard, t, 1440)) <= yardInventory_10.get(yard))
            
# 0

In [27]:
## Inventory Balance at Yard / Tail Track (10 Car Breakable) 


# Update 10 Car Inventory. TS_abc = 1. Trains arriving. 
model.addConstrs((inventoryYard.get((y, l, t)) == inventoryYard.get((y, l, t-1)) 
                  + segmentTrain.get((seg, d, t)) 
                 for y, l, seg, d, t in [(y, l, seg, d, t) 
                 for y in yard_tt for seg in segment if segmentYard_Relation.get(a)[1] == y 
                 for d in day for t in time if t > 300 for l in typeTrain if l == 10]))


# Update 10 Car Inventory. TS_abc = 1. Trains departing. 
model.addConstrs((inventoryYard.get((y, l, t)) == inventoryYard.get((y, l, t-1)) 
                  - segmentTrain.get((seg, d, t)) 
                  for y, l, seg, d, t in [(y, l, seg, d, t) for y in yard_tt 
                  for seg in segment if segmentYard_Relation.get(a)[0] == y 
                  for d in day for t in time if t > 300 for l in typeTrain if l == 10]))

0

0

In [28]:
## Inventory Balance at Yard / Tail Track (5 Car Makable) 


# Update 5 Car Inventory. TS_abc = 0. Trains arriving. 
model.addConstrs((inventoryYard.get((y, l, t)) == inventoryYard.get((y, l, t-1)) 
                  + (1 - segmentTrain.get((seg, d, t))) 
                 for y, l, seg, d, t in [(y, l, seg, d, t) 
                 for y in yard_tt for seg in segment if segmentYard_Relation.get(a)[1] == y 
                 for d in day for t in index if t > 300 for l in typeTrain if l == 5]))


# Update 5 Car Inventory. TS_abc = 0. Trains departing. 
model.addConstrs((inventoryYard.get((y, l, t)) == inventoryYard.get((y, l, t-1)) 
                  - (1 - segmentTrain.get((seg, d, t))) 
                  for y, l, seg, d, t in [(y, l, seg, d, t) 
                  for y in yard_tt for seg in segment if segmentYard_Relation.get(a)[0] == y 
                  for d in day for t in index if t > 300 for l in typeTrain if l == 5]))

0

0

In [16]:
# NO M/B at Hayward/WarmSprings, Concord, or SFO. Have this as a constraint. How could I do this? 
# (Similar approach to the Obj approach)

no_makebreak_yards = ["WarmSprings, SFO"]
# perhaps use these as indicators to let the model know that we can't make/break at these yards. 
# A train segment that arrives as a 1 must leave as a 1 in the next train segment and vice a versa. 

# model.addConstrs((segmentTrain.get((seg_departure, d, i)) == segmentTrain.get((seg_departing, d, i))))
    


In [ ]:
# Force train segment to change 
# if making/breaking is not possible and if inventory fulfillment is not possible at the yard 

# Train segment must be zero if the yard of departure does not have enough inventory for fulfillment or for making trains
# model.addConstrs(segmentTrain.get((seg, d, i)) == 0 
#                  for seg, d, i in [(seg, d, i) 
#                                           for seg in segment if (inventoryYard.get((segmentYard_Relation.get(seg)[0], 10, c)) == 0 
#                                                             and inventoryYard.get((segmentYard_Relation.get(seg)[0], 5, c)) <= 1)
#                                           for d in day
#                                           for i in index])


# These constraints will not work because they are not independent from eachother. These constraints are covered by the objective function anyhow..

# # Segment can not run a long train if a make is not possible at its departure yard or inventory. If it cant be 1 than it must be 0.
# # Less than or equal to 1 5 car train at yard 
# # 0 10 car trains 
# # AND TRYING TO MAKE 
model.addConstrs(segmentTrain.get((a, b, c)) == 0 for a, b, c in [(a, b, c) 
                  for a in segment if (inventoryYard.get((segmentYard_Relation.get(a)[0], 5, c)) <= 1 and 
                                       inventoryYard.get((segmentYard_Relation.get(a)[0], 10, c)) == 0)
                                       for b in day 
                                       for c in index])

# # Segment can not run a short train if a break is not possible at its departure yard or inventory. If it cant be 0 than it must be 1.
# # 0 breakable cars at that yard 
# # 0 5 car trains at that yard 
# # AND TRYING TO BREAK
model.addConstrs(segmentTrain.get((a, b, c)) == 1 for a, b, c in [(a, b, c) 
                  for a in segment if (inventoryYard.get((segmentYard_Relation.get(a)[0], 5, c)) == 0 and 
                                       inventoryYard.get((segmentYard_Relation.get(a)[0], 10, c)) == 0) 
                                       for b in day 
                                       for c in index])
0


### Solve 

- Model is infeasible when 5 car inventory balance constraints and 10 car inventory balance constraints are both introduced. 
    - Model has solution when only one of the set of constraints is introduced. 
    - Both sets of constraints combined produces a solution when no other constraints are introduced additionally. 

In [17]:
model.update()
model.optimize()


Optimize a model with 289588 rows, 84504 columns and 836788 nonzeros
Variable types: 15974 continuous, 68530 integer (68530 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 289588 rows and 84504 columns
Presolve time: 0.54s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.91 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
